<h1>Analysis of a specific matchup<h1>

0. Imports

In [96]:
import pandas as pd
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.static import teams
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import nba_analysis_helpers as methods
from plotly.subplots import make_subplots
from nba_api.stats.endpoints import leaguedashteamstats

<h2>Team parameters<h2>

In [97]:
team_1 = "Phoenix Suns"
team_2 = "Miami Heat"

<h2>Make prepared logs for the teams<h2>

In [98]:
log_1 = methods.prepare_data(methods.get_team_log(team_1))
log_2 = methods.prepare_data(methods.get_team_log(team_2))


<h3>Here some plot about te teams ponts:<h3>

In [99]:
methods.make_hist_subplot(log_1,log_2,team_1,team_2,'PTS')

In [100]:
log_1['MATCHUP'] = methods.clean_matchup(log_1)
log_2['MATCHUP'] = methods.clean_matchup(log_2)

In [101]:
log_1_by_opponents = methods.scores_by_opponent(log_1, team_2)
log_2_by_opponents = methods.scores_by_opponent(log_2, team_1)

KeyError: 'MIA'

In [ ]:
from IPython.display import display_html 


df1_styler = log_1_by_opponents.style.set_table_attributes("style='display:inline'").set_caption('df1')
df2_styler = log_2_by_opponents.style.set_table_attributes("style='display:inline'").set_caption('df2')
df2_t_styler = log_2_by_opponents.T.style.set_table_attributes("style='display:inline'").set_caption('df2_t')

display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)
# display_html(df1_styler._repr_html_()+df2_styler._repr_html_() +df2_t_styler._repr_html_(), raw=True)

,MIL
count,1.000000
min,125.000000
max,125.000000
mean,125.000000
median,125.000000
,CHA
count,1.000000
min,127.000000
max,127.000000
mean,127.000000


<h2>Lets see the season statics ot the two teams<h2>

In [102]:
team_1_ID = teams.find_teams_by_full_name(team_1)[0]['id']
team_2_ID = teams.find_teams_by_full_name(team_2)[0]['id']

teams_stats = pd.DataFrame(
    [leaguedashteamstats.LeagueDashTeamStats(team_id_nullable=team_1_ID).get_normalized_dict()['LeagueDashTeamStats'][0],
    leaguedashteamstats.LeagueDashTeamStats(team_id_nullable=team_2_ID).get_normalized_dict()['LeagueDashTeamStats'][0]]
    )

teams_stats['TEAM_NAME'] = teams_stats['TEAM_NAME'].map(lambda x: 'Los Angeles Clippers' if x == 'LA Clippers' else x)
teams_stats[['TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'FG_PCT', 'FG3_PCT','FT_PCT', 'PLUS_MINUS', 'CFID' ]]

,TEAM_NAME,GP,W,L,W_PCT,FG_PCT,FG3_PCT,FT_PCT,PLUS_MINUS,CFID
0,Phoenix Suns,38,30,8,0.789,0.475,0.369,0.799,300.0,10
1,Miami Heat,39,24,15,0.615,0.458,0.373,0.797,147.0,10


<h3>Lets visualize it:<h3>

In [103]:
methods.make_stat_comparison_subplot(teams_stats,team_1,team_2)

<h2>Lets make a train dataset to make a ML model to predict games<h2>

<h3>Table from the team stats by each game<h3>

In [104]:
from nba_api.stats.endpoints import teamgamelog
team_1_ID = teams.find_teams_by_full_name(team_1)[0]['id']
team_games = teamgamelog.TeamGameLog(team_id=team_1_ID).get_normalized_dict()["TeamGameLog"]

team_games = pd.DataFrame(team_games)
print(team_games.columns)
team_games.head()

Index(['Team_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'W', 'L', 'W_PCT',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS'],
      dtype='object')


,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612756,0022100581,"JAN 06, 2022",PHX vs. LAC,W,30,8,0.789,240,44,...,0.800,21,42,63,22,5,7,8,20,106
1,1610612756,0022100565,"JAN 04, 2022",PHX @ NOP,W,29,8,0.784,240,46,...,0.700,5,37,42,32,7,5,10,24,123
2,1610612756,0022100549,"JAN 02, 2022",PHX @ CHA,W,28,8,0.778,240,52,...,1.000,14,39,53,36,15,4,17,21,133
3,1610612756,0022100529,"DEC 31, 2021",PHX @ BOS,L,27,8,0.771,240,41,...,0.938,15,22,37,20,10,2,9,21,108
4,1610612756,0022100522,"DEC 29, 2021",PHX vs. OKC,W,27,7,0.794,240,41,...,0.750,18,38,56,22,7,7,14,15,115


In [105]:
#címke tömb
labels = team_games['WL']

#ebbe a táblába csak a statokat rakjuk
train_stats = team_games[['W', 'L', 'W_PCT']]

#hozzá kell adni egy plusz sort az elejére
train_stats = train_stats.append({'W':0,'L':0,'W_PCT':0}, ignore_index=True)
#meg kell fordítani a sorrendet
train_stats = train_stats[::-1]
#el kell dobni az utolsó sort
train_stats = train_stats.drop(0)
#indexeket a megfordítás miatt újra kell állítani
train_stats = train_stats.reset_index(drop=True)

#ide azokat amiket el kell csúsztatni
train = team_games[['GAME_DATE', 'MATCHUP']][::-1]
#indexeket a megfordítás miatt újra kell állítani
train = train.reset_index(drop=True)

#össze kell rakni a két elkészült táblákat 
train = pd.concat([train,train_stats], axis=1)

#át kell alakítani a matchupot úgy hogy értelmesebb, használhatóbb legyen
train['MATCHUP'] = train['MATCHUP'].map(lambda x: x.split(' ')[2])

train['GAME_DATE'] = pd.to_datetime(train['GAME_DATE'], format="%b %d, %Y")
train['day_in_month'] = train['GAME_DATE'].map(lambda x: int(x.strftime("%d")))
train['day_in_week'] = train['GAME_DATE'].map(lambda x: int(x.strftime("%w")))
train = train.drop('GAME_DATE', axis=1)

train.head()

,MATCHUP,W,L,W_PCT,day_in_month,day_in_week
0,DEN,0,0,0.000,20,3
1,LAL,0,1,0.000,22,5
2,POR,1,1,0.500,23,6
3,SAC,1,2,0.333,27,3
4,CLE,1,3,0.250,30,6


<h3>Table from the team lineups<h3>

<h4>Some Preprocessing<h4>

In [106]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
#TODO így nem lesz benne minden csapat, csak akik ellen már játszottak, midnet bele kéne rakni
encoder.fit(train.MATCHUP.unique())
encoder.classes_

array(['ATL', 'BKN', 'BOS', 'CHA', 'CLE', 'DAL', 'DEN', 'DET', 'GSW',
       'HOU', 'LAC', 'LAL', 'MEM', 'MIN', 'NOP', 'NYK', 'OKC', 'POR',
       'SAC', 'SAS', 'WAS'], dtype=object)

In [107]:
train['MATCHUP'] = encoder.transform(train.MATCHUP)

In [108]:
train.head()

,MATCHUP,W,L,W_PCT,day_in_month,day_in_week
0,6,0,0,0.000,20,3
1,11,0,1,0.000,22,5
2,17,1,1,0.500,23,6
3,18,1,2,0.333,27,3
4,4,1,3,0.250,30,6


<h4>Split to train and test sets<h4>

In [109]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.3, random_state=12)

<h4>Make the model<h4>

In [110]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train,y_train)

model

GaussianNB()

<h4>Lets test on the test dataset<h4>

In [111]:
model.predict(X_test)

array(['W', 'W', 'W', 'W', 'W', 'W', 'L', 'W', 'L', 'W', 'L', 'W'],
      dtype='<U1')

In [112]:
model.score(X_test,y_test)

0.75

In [113]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=4)
scores

array([0.71428571, 0.71428571, 0.66666667, 0.5       ])

<h4>Now, we can predict the next game, firstly need to create a compatible table about the current stats<h4>

In [114]:
current_stats = team_games.iloc[0][['GAME_DATE','MATCHUP','W','L','W_PCT']]
current_stats['GAME_DATE'] = pd.to_datetime(current_stats['GAME_DATE'], format="%b %d, %Y")
current_stats['day_in_month'] = int(current_stats['GAME_DATE'].strftime("%d"))
current_stats['day_in_week'] = current_stats['GAME_DATE'].strftime("%w")
current_stats = current_stats.drop('GAME_DATE')

opponent_abb = teams.find_teams_by_full_name(team_2)[0]['abbreviation']

matchup = encoder.transform([opponent_abb])
w = [current_stats['W']]
l = [current_stats['L']]
wpct = [current_stats['W_PCT']]
month = [current_stats['day_in_month']]
week = [current_stats['day_in_week']]

current_stats = pd.DataFrame({'MATCHUP': matchup,'W': w, 'L': l, 'W_PCT': wpct, 'day_in_month':month, 'day_in_week': week})
current_stats

ValueError: y contains previously unseen labels: 'MIA'

In [ ]:
model.predict(current_stats)

array(['W'], dtype='<U1')